# Prerequirements

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Preprocess the raw data

## Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select significant features (columns)
Perform some feature selection based upon physical characteristics of the candidates to exoplanets

In [12]:
# Set features. This will also be used as x values.
# KOI = Kepler Objects of Interest
X = df[['koi_fpflag_nt', #A KOI whose light curve is not consistent with that of a transiting planet.
        'koi_fpflag_ss',
        'koi_fpflag_co',
        'koi_fpflag_ec',
        'koi_period',
        'koi_period_err1',
        'koi_duration',
        'koi_duration_err1',
        'koi_depth',
        'koi_depth_err1',
        'koi_prad',
        'koi_prad_err1',
        'koi_prad_err2',
        'koi_teq',
        'koi_insol',
        'koi_insol_err1',
        'koi_insol_err2',
        'koi_steff',
        'koi_steff_err1',
        'koi_steff_err2',
        'koi_slogg',
        'koi_slogg_err1',
        'koi_slogg_err2',
        'koi_srad',
        'koi_srad_err1',
        'koi_srad_err2']]
X.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_duration,koi_duration_err1,koi_depth,koi_depth_err1,...,koi_insol_err2,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2
0,0,0,0,0,54.418383,2.479000e-04,4.50700,0.11600,874.8,35.5,...,-1.62,5455,81,-81,4.467,0.064,-0.096,0.927,0.105,-0.061
1,0,1,0,0,19.899140,1.490000e-05,1.78220,0.03410,10829.0,171.0,...,-10.49,5853,158,-176,4.544,0.044,-0.176,0.868,0.233,-0.078
2,0,1,0,0,1.736952,2.630000e-07,2.40641,0.00537,8079.2,12.8,...,-230.35,5805,157,-174,4.564,0.053,-0.168,0.791,0.201,-0.067
3,0,0,0,0,2.525592,3.760000e-06,1.65450,0.04200,603.3,16.9,...,-314.24,6031,169,-211,4.438,0.070,-0.210,1.046,0.334,-0.133
4,0,0,0,0,4.134435,1.050000e-05,3.14020,0.06730,686.0,18.7,...,-136.70,6046,189,-232,4.486,0.054,-0.229,0.972,0.315,-0.105


Use `koi_disposition` for the y values

In [4]:
y = df["koi_disposition"]
y.head()

0         CONFIRMED
1    FALSE POSITIVE
2    FALSE POSITIVE
3         CONFIRMED
4         CONFIRMED
Name: koi_disposition, dtype: object

## Spliting the data into training and testing data.

In [5]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_duration,koi_duration_err1,koi_depth,koi_depth_err1,...,koi_insol_err2,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2
6122,0,0,0,0,6.768901,7.380000e-05,3.61600,0.3060,123.1,13.2,...,-103.87,5737,154,-171,4.327,0.153,-0.187,1.125,0.310,-0.207
6370,0,1,0,1,0.733726,6.060000e-06,2.30900,0.2820,114.6,9.3,...,-677.78,5855,158,-175,4.578,0.033,-0.187,0.797,0.211,-0.056
2879,1,0,0,0,7.652707,6.540000e-05,79.89690,0.0000,641.1,42.9,...,-64.34,6328,151,-189,4.481,0.050,-0.200,0.963,0.290,-0.097
107,0,0,0,0,7.953547,1.910000e-05,2.63120,0.0595,875.4,25.3,...,-10.12,4768,76,-85,4.536,0.056,-0.016,0.779,0.023,-0.049
29,0,0,0,0,4.959319,5.150000e-07,2.22739,0.0075,9802.0,20.0,...,-93.21,5712,77,-77,4.359,0.110,-0.110,1.082,0.173,-0.130


## Scaling the data

Scaling the data using the MinMaxScaler

In [6]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
X.shape

(6991, 26)

In [8]:
y.shape

(6991,)

# Original Model

In [9]:
# Create a random forest classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)

# Train the Model = Fit the Model with Trained Data

In [10]:
model.fit(X_train_scaled, y_train)

RandomForestClassifier(n_estimators=200)

# Score the Model with Test Data

In [11]:
model_training_score = round(model.score(X_train_scaled, y_train)*100,3)
model_accuracy = round(model.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {model_training_score}%")
print(f"Testing Data Score: {model_accuracy}%")

Training Data Score: 100.0%
Testing Data Score: 89.359%


# Hyperparameter Tuning (temporay Model)

Using `GridSearchCV` to tune the model's parameters

In [14]:
# Create the GridSearchCV model from scratch only to find best/tuned parameters
from sklearn.model_selection import GridSearchCV
# model_withGridSerach = RandomForestClassifier(random_state=42)

In [15]:
param_grid = {
    'n_estimators': [200, 600, 1200, 1400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [14, 15, 16, 17, 18, None]
}
grid_model = GridSearchCV(model, param_grid, cv=5, verbose=3, n_jobs=-1)

# Train the model with GridSearch
_ = grid_model.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [16]:
# Find Tuned parameters
max_features = grid_model.best_params_['max_features']
n_estimators = grid_model.best_params_['n_estimators']
max_depth = grid_model.best_params_['max_depth']
criterion = 'entropy'

# Tuned Model

In [17]:
# Tuned model based upon best parameters previously found
tuned_model = RandomForestClassifier(max_features=max_features, n_estimators=n_estimators, 
                                     criterion=criterion, max_depth=max_depth, random_state=42)

In [19]:
tuned_model.fit(X_train_scaled, y_train)

tuned_training_score = round(tuned_model.score(X_train_scaled, y_train)*100,3)
tuned_accuracy = round(tuned_model.score(X_test_scaled, y_test)*100,3)

print(f"Training Data Score: {tuned_training_score} %")
print(f"Testing Data Score: {tuned_accuracy} %")

Training Data Score: 98.436 %
Testing Data Score: 89.531 %


# Save the Model

In [21]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Models/RandomForest-LuisRojas.sav'
joblib.dump(tuned_model, filename)

['LuisRojas.sav']